## Table: fighter_instances

### First bout scrape

In [17]:
first_bout = fightCard_df.loc[0]

In [18]:
bout_link = fightCard_df.loc[0].link
bout_url = 'https://www.tapology.com'+bout_link
response = requests.get(bout_url, headers=headers)
bout_soup = BeautifulSoup(response.text, 'html.parser')

Let's see if we can find a class for the list we want in this page, we'll use chrome developer's tools.

Looking at the page, I found that the info I want is under the boutComparisonTable class. So let's pull that first.

In [19]:
bout_table = bout_soup.find_all(class_='boutComparisonTable')

Actually, I can just pull the table in with pd so let's try that first.

In [20]:
tables = pd.read_html(response.content)

In [21]:
len(tables)

3

In [22]:
tables[0]

,0,1,2,3,4
0,18-3-0,NaN,Pro Record At Fight,NaN,19-4-1
1,Climbed to 19-3,NaN,Record After Fight,NaN,Fell to 19-5
2,+150 (Slight Underdog),NaN,Betting Odds,NaN,-185 (Slight Favorite)
3,Brazil,NaN,Nationality,NaN,United States
4,"Boca Raton, Florida",NaN,Fighting out of,NaN,"St. Louis, Missouri"
5,"33 years, 10 months, 3 days",NaN,Age at Fight,NaN,"38 years, 1 month, 6 days"
6,170.5 lbs (77.3 kgs),NaN,Weigh-In Result,NaN,170.5 lbs (77.3 kgs)
7,"5'10"" (178cm)",NaN,Height,NaN,"5'9"" (176cm)"
8,"71.0"" (180cm)",NaN,Reach,NaN,"74.0"" (188cm)"
9,Blackzilians,NaN,Gym,NaN,American Top Team


Now we should be able to pivot it.

In [23]:
bout_stats = tables[0]
bout_stats.columns

Int64Index([0, 1, 2, 3, 4], dtype='int64')

In [24]:
bout_stats_pivot = pd.DataFrame([list(bout_stats[0]), list(bout_stats[4])])

In [25]:
bout_stats_pivot.columns = list(bout_stats[2])

In [26]:
bout_stats_pivot.reset_index(inplace=True, drop=True)

In [27]:
bout_stats_pivot

,Pro Record At Fight,Record After Fight,Betting Odds,Nationality,Fighting out of,Age at Fight,Weigh-In Result,Height,Reach,Gym
0,18-3-0,Climbed to 19-3,+150 (Slight Underdog),Brazil,"Boca Raton, Florida","33 years, 10 months, 3 days",170.5 lbs (77.3 kgs),"5'10"" (178cm)","71.0"" (180cm)",Blackzilians
1,19-4-1,Fell to 19-5,-185 (Slight Favorite),United States,"St. Louis, Missouri","38 years, 1 month, 6 days",170.5 lbs (77.3 kgs),"5'9"" (176cm)","74.0"" (188cm)",American Top Team


Now let's test it in function form.

In [28]:
fighter_instances = src.create_fighter_instances_table(response.content, bout_link)
fighter_instances

,Pro Record At Fight,Record After Fight,Betting Odds,Nationality,Fighting out of,Age at Fight,Weigh-In Result,Height,Reach,Gym,fighter_link,bout_link,instance_id
0,18-3-0,Climbed to 19-3,+150 (Slight Underdog),Brazil,"Boca Raton, Florida","33 years, 10 months, 3 days",170.5 lbs (77.3 kgs),"5'10"" (178cm)","71.0"" (180cm)",Blackzilians,/fightcenter/fighters/31168-gilbert-burns-durinho,/fightcenter/bouts/501343-ufc-fight-night-tyro...,/fightcenter/bouts/501343-ufc-fight-night-tyro...
1,19-4-1,Fell to 19-5,-185 (Slight Favorite),United States,"St. Louis, Missouri","38 years, 1 month, 6 days",170.5 lbs (77.3 kgs),"5'9"" (176cm)","74.0"" (188cm)",American Top Team,/fightcenter/fighters/tyron-woodley-t-wood,/fightcenter/bouts/501343-ufc-fight-night-tyro...,/fightcenter/bouts/501343-ufc-fight-night-tyro...


Now I need to add in the links, so maybe I can use the code I wrote to get the links from the other links functions I made. First I'll find where the table is located.

In [29]:
name_elem = bout_soup.find(class_="fighterNames botPad clearfix")
name_elem

<div class="fighterNames botPad clearfix">
<p>
<span class="fName left">
<a href="/fightcenter/fighters/31168-gilbert-burns-durinho">Gilbert Burns</a>
</span>
<span class="fName right">
<a href="/fightcenter/fighters/tyron-woodley-t-wood">Tyron Woodley</a>
</span>
</p>
<p>
<span class="fName left">"Durinho"</span>
<span class="fName right">"The Chosen One"</span>
</p>
</div>

In [30]:
name_elem = name_elem.find_all('a')
name_elem

[<a href="/fightcenter/fighters/31168-gilbert-burns-durinho">Gilbert Burns</a>,
 <a href="/fightcenter/fighters/tyron-woodley-t-wood">Tyron Woodley</a>]

In [31]:
links = [name.get('href') for name in name_elem]
links

['/fightcenter/fighters/31168-gilbert-burns-durinho',
 '/fightcenter/fighters/tyron-woodley-t-wood']

Now I'll add this onto my dataframe.

In [32]:
fighter_instances['fighter_link'] = links
fighter_instances


,Pro Record At Fight,Record After Fight,Betting Odds,Nationality,Fighting out of,Age at Fight,Weigh-In Result,Height,Reach,Gym,fighter_link,bout_link,instance_id
0,18-3-0,Climbed to 19-3,+150 (Slight Underdog),Brazil,"Boca Raton, Florida","33 years, 10 months, 3 days",170.5 lbs (77.3 kgs),"5'10"" (178cm)","71.0"" (180cm)",Blackzilians,/fightcenter/fighters/31168-gilbert-burns-durinho,/fightcenter/bouts/501343-ufc-fight-night-tyro...,/fightcenter/bouts/501343-ufc-fight-night-tyro...
1,19-4-1,Fell to 19-5,-185 (Slight Favorite),United States,"St. Louis, Missouri","38 years, 1 month, 6 days",170.5 lbs (77.3 kgs),"5'9"" (176cm)","74.0"" (188cm)",American Top Team,/fightcenter/fighters/tyron-woodley-t-wood,/fightcenter/bouts/501343-ufc-fight-night-tyro...,/fightcenter/bouts/501343-ufc-fight-night-tyro...


I also need a way to refer to these specific elements so I'm going to create an instance ID that is made up of the bout_link and the fighter_link combined.

In [33]:
fighter_instances['instance_id'] = fighter_instances['bout_link'] + fighter_instances['fighter_link']
fighter_instances

,Pro Record At Fight,Record After Fight,Betting Odds,Nationality,Fighting out of,Age at Fight,Weigh-In Result,Height,Reach,Gym,fighter_link,bout_link,instance_id
0,18-3-0,Climbed to 19-3,+150 (Slight Underdog),Brazil,"Boca Raton, Florida","33 years, 10 months, 3 days",170.5 lbs (77.3 kgs),"5'10"" (178cm)","71.0"" (180cm)",Blackzilians,/fightcenter/fighters/31168-gilbert-burns-durinho,/fightcenter/bouts/501343-ufc-fight-night-tyro...,/fightcenter/bouts/501343-ufc-fight-night-tyro...
1,19-4-1,Fell to 19-5,-185 (Slight Favorite),United States,"St. Louis, Missouri","38 years, 1 month, 6 days",170.5 lbs (77.3 kgs),"5'9"" (176cm)","74.0"" (188cm)",American Top Team,/fightcenter/fighters/tyron-woodley-t-wood,/fightcenter/bouts/501343-ufc-fight-night-tyro...,/fightcenter/bouts/501343-ufc-fight-night-tyro...


Function testing:

In [34]:
src.create_fighter_instances_table(response.content, bout_link)

,Pro Record At Fight,Record After Fight,Betting Odds,Nationality,Fighting out of,Age at Fight,Weigh-In Result,Height,Reach,Gym,fighter_link,bout_link,instance_id
0,18-3-0,Climbed to 19-3,+150 (Slight Underdog),Brazil,"Boca Raton, Florida","33 years, 10 months, 3 days",170.5 lbs (77.3 kgs),"5'10"" (178cm)","71.0"" (180cm)",Blackzilians,/fightcenter/fighters/31168-gilbert-burns-durinho,/fightcenter/bouts/501343-ufc-fight-night-tyro...,/fightcenter/bouts/501343-ufc-fight-night-tyro...
1,19-4-1,Fell to 19-5,-185 (Slight Favorite),United States,"St. Louis, Missouri","38 years, 1 month, 6 days",170.5 lbs (77.3 kgs),"5'9"" (176cm)","74.0"" (188cm)",American Top Team,/fightcenter/fighters/tyron-woodley-t-wood,/fightcenter/bouts/501343-ufc-fight-night-tyro...,/fightcenter/bouts/501343-ufc-fight-night-tyro...


## Table: bouts (addition)

Once I got to the bout page I realized that there was some info that I would like to get but was not available on the initial page. I'm going to grab it from the bout page itself and add it to the dataframe. First let's open up our first bout page. I also want to make sure I have my bouts dataframe available so that I can add to it.

In [40]:
df_events = previous_ufc
df_bouts = src.get_initial_bout_df()
df_bouts.head(5)

,method,length,order,fighter_0,bout_type,weight_class,scheduled_rounds,fighter_1,link,event_link
0,"Decision, Unanimous","5 Rounds, 25:00 Total",11,Gilbert Burns,Main Event,170,5 x 5,Tyron Woodley,/fightcenter/bouts/501343-ufc-fight-night-tyro...,/fightcenter/events/69127-ufc-fight-night
1,"Decision, Split","3 Rounds, 15:00 Total",10,Augusto Sakai,Co-Main Event,265,3 x 5,Blagoy Ivanov,/fightcenter/bouts/501441-ufc-fight-night-blag...,/fightcenter/events/69127-ufc-fight-night
2,"Decision, Unanimous","3 Rounds, 15:00 Total",9,Billy Quarantillo,Main Card,150,3 x 5,Spike Carlyle,/fightcenter/bouts/502990-ufc-fight-night-bill...,/fightcenter/events/69127-ufc-fight-night
3,"Submission, Rear Naked Choke","3:26 Round 2 of 3, 8:26 Total",8,Roosevelt Roberts,Main Card,155,3 x 5,Brok Weaver,/fightcenter/bouts/502972-ufc-fight-night-roos...,/fightcenter/events/69127-ufc-fight-night
4,"Submission, Kneebar",2:36 Round 1 of 3,7,Mackenzie Dern,Main Card,115,3 x 5,Hannah Cifers,/fightcenter/bouts/500998-ufc-fight-night-mack...,/fightcenter/events/69127-ufc-fight-night


According to the database schema I created, I want the following categories, The ones with check marks are the ones that my initial bout datafram already contains:

- [x] link
- [x] method
- [x] length 
- [x] order
- [x] fighter_0
- [x] fighter_1
- [x] bout_type
- [x] scheduled_rounds
- [x] weight_class
- [ ] pro_am
- [ ] referee
- [x] event_link

So in order to get the pro_am and the referee info, I need to follow the bout link to and pull that info from a table on that page. Let's pull up the bout page.

In [94]:
response = requests.get(bout_url, headers=headers)
bout_soup = BeautifulSoup(response.content, 'html.parser')

Looking at the web page I see that there is a div element with a certain class that contains the table I'm looking for.

In [52]:
info_div = bout_soup.find('div', class_='details details_with_poster clearfix')
missing_info_elem = info_div.find('ul')

This looks similar to the list I parsed above so maybe I can reuse the same list parsing functions.

In [56]:
info_list = missing_info_elem.get_text().split('\n\n\n')#turn it into text and split into list
    
info_list = src.clean_info_list(info_list)
    
info_list

['Bout Information',
 'Event:',
 'UFC Fight Night: Woodley vs. Burns',
 'Date:',
 'Saturday 05.30.2020 at 06:00 PM ET',
 'Referee:',
 'Herb Dean',
 'Venue:',
 'UFC APEX',
 'Enclosure:',
 'Octagon',
 'Location:',
 'Las Vegas, Nevada, United States',
 'Bout Billing:',
 'Main Event',
 '(fight 11 of 11)',
 'Pro/Am:',
 'Professional',
 'Weight:',
 '170 lbs (77.1 kg)',
 'TV Commentary:',
 'Brendan Fitzgerald, Michael Bisping, Daniel Cormier',
 'Broadcast:',
 'Aired Live on Main Card',
 'Post-Fight Interviewer:',
 'Daniel Cormier',
 'Woodley Total Disclosed Pay:',
 '$200,000',
 'Burns Total Disclosed Pay:',
 '$218,000']

In [66]:
src.html_list_to_df(missing_info_elem)

,Event:,Date:,Referee:,Venue:,Enclosure:,Location:,Bout Billing:,(fight 11 of 11),Professional,170 lbs (77.1 kg),"Brendan Fitzgerald, Michael Bisping, Daniel Cormier",Aired Live on Main Card,Daniel Cormier,"$200,000",first_elem
0,UFC Fight Night: Woodley vs. Burns,Saturday 05.30.2020 at 06:00 PM ET,Herb Dean,UFC APEX,Octagon,"Las Vegas, Nevada, United States",Main Event,Pro/Am:,Weight:,TV Commentary:,Broadcast:,Post-Fight Interviewer:,Woodley Total Disclosed Pay:,Burns Total Disclosed Pay:,Bout Information


The function I built does not work on this list for some readon.

In [68]:
missing_info_elem.get_text()

'\nBout Information\n\nEvent:\nUFC Fight Night: Woodley vs. Burns\n\n\nDate:\nSaturday 05.30.2020 at 06:00 PM ET\n\n\nReferee:\nHerb Dean\n\n\nVenue:\nUFC APEX\n\n\nEnclosure:\nOctagon\n\n\nLocation:\nLas Vegas, Nevada, United States\n\n\nBout Billing:\nMain Event\n(fight 11 of 11)\n\n\nPro/Am:\nProfessional\n\n\n\nWeight:\n170 lbs (77.1 kg)\n\n\n\nTV Commentary:\nBrendan Fitzgerald, Michael Bisping, Daniel Cormier\n\n\nBroadcast:\nAired Live on Main Card\n\n\nPost-Fight Interviewer:\nDaniel Cormier\n\n\nWoodley Total Disclosed Pay:\n$200,000\n\n\nBurns Total Disclosed Pay:\n$218,000\n\n\nEvent Links:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBout Links:\n\n\n\n\n\n\n\n\n\n'

The function html_list_to_df is confused by the fact that there is a new line between event and (fight 11 of 11).

In [79]:
info_list = missing_info_elem.get_text().split('\n\n\n')#turn it into text and split into list
new_list = [] 
for item in info_list:
    #fields with no info do not have a colon followed by a new line, so i will take those out
    if ':\n' in item: 
        new_list.append(item)   
#this creates a list with the info separated by empty string elements
info_list = '\n\n'.join(new_list).split('\n\n') 
#this removes those empty string elements
info_list = list(filter(lambda item: item != '', info_list)) 

In [80]:
info_list

['\nBout Information',
 'Event:\nUFC Fight Night: Woodley vs. Burns',
 'Date:\nSaturday 05.30.2020 at 06:00 PM ET',
 'Referee:\nHerb Dean',
 'Venue:\nUFC APEX',
 'Enclosure:\nOctagon',
 'Location:\nLas Vegas, Nevada, United States',
 'Bout Billing:\nMain Event\n(fight 11 of 11)',
 'Pro/Am:\nProfessional',
 '\nWeight:\n170 lbs (77.1 kg)',
 '\nTV Commentary:\nBrendan Fitzgerald, Michael Bisping, Daniel Cormier',
 'Broadcast:\nAired Live on Main Card',
 'Post-Fight Interviewer:\nDaniel Cormier',
 'Woodley Total Disclosed Pay:\n$200,000',
 'Burns Total Disclosed Pay:\n$218,000']

In [81]:
info_list.pop(0)
info_list = [elem.split(':\n') for elem in info_list]

In [82]:
info_list

[['Event', 'UFC Fight Night: Woodley vs. Burns'],
 ['Date', 'Saturday 05.30.2020 at 06:00 PM ET'],
 ['Referee', 'Herb Dean'],
 ['Venue', 'UFC APEX'],
 ['Enclosure', 'Octagon'],
 ['Location', 'Las Vegas, Nevada, United States'],
 ['Bout Billing', 'Main Event\n(fight 11 of 11)'],
 ['Pro/Am', 'Professional'],
 ['\nWeight', '170 lbs (77.1 kg)'],
 ['\nTV Commentary', 'Brendan Fitzgerald, Michael Bisping, Daniel Cormier'],
 ['Broadcast', 'Aired Live on Main Card'],
 ['Post-Fight Interviewer', 'Daniel Cormier'],
 ['Woodley Total Disclosed Pay', '$200,000'],
 ['Burns Total Disclosed Pay', '$218,000']]

In [85]:
src.html_list_to_df(missing_info_elem)

[['Event', 'UFC Fight Night: Woodley vs. Burns'], ['Date', 'Saturday 05.30.2020 at 06:00 PM ET'], ['Referee', 'Herb Dean'], ['Venue', 'UFC APEX'], ['Enclosure', 'Octagon'], ['Location', 'Las Vegas, Nevada, United States'], ['Bout Billing', 'Main Event\n(fight 11 of 11)'], ['Pro/Am', 'Professional'], ['\nWeight', '170 lbs (77.1 kg)'], ['\nTV Commentary', 'Brendan Fitzgerald, Michael Bisping, Daniel Cormier'], ['Broadcast', 'Aired Live on Main Card'], ['Post-Fight Interviewer', 'Daniel Cormier'], ['Woodley Total Disclosed Pay', '$200,000'], ['Burns Total Disclosed Pay', '$218,000']]


,Event,Date,Referee,Venue,Enclosure,Location,Bout Billing,Pro/Am,\nWeight,\nTV Commentary,Broadcast,Post-Fight Interviewer,Woodley Total Disclosed Pay,Burns Total Disclosed Pay,first_elem
0,UFC Fight Night: Woodley vs. Burns,Saturday 05.30.2020 at 06:00 PM ET,Herb Dean,UFC APEX,Octagon,"Las Vegas, Nevada, United States",Main Event\n(fight 11 of 11),Professional,170 lbs (77.1 kg),"Brendan Fitzgerald, Michael Bisping, Daniel Co...",Aired Live on Main Card,Daniel Cormier,"$200,000","$218,000",\nBout Information


Basically I modified my get_missing_event_info function to work on bout pages as well.

In [99]:
missing_bout_info = src.get_missing_bout_info(bout_soup, first_link)

In [100]:
fightCard_df.join(missing_bout_info.set_index('link'), on='link')

,method,length,order,fighter_0,bout_type,weight_class,scheduled_rounds,fighter_1,link,event_link,referee,pro_am
0,"Decision, Unanimous","5 Rounds, 25:00 Total",11,Gilbert Burns,Main Event,170,5 x 5,Tyron Woodley,/fightcenter/bouts/501343-ufc-fight-night-tyro...,/fightcenter/events/69127-ufc-fight-night,Herb Dean,Professional
1,"Decision, Split","3 Rounds, 15:00 Total",10,Augusto Sakai,Co-Main Event,265,3 x 5,Blagoy Ivanov,/fightcenter/bouts/501441-ufc-fight-night-blag...,/fightcenter/events/69127-ufc-fight-night,NaN,NaN
2,"Decision, Unanimous","3 Rounds, 15:00 Total",9,Billy Quarantillo,Main Card,150,3 x 5,Spike Carlyle,/fightcenter/bouts/502990-ufc-fight-night-bill...,/fightcenter/events/69127-ufc-fight-night,NaN,NaN
3,"Submission, Rear Naked Choke","3:26 Round 2 of 3, 8:26 Total",8,Roosevelt Roberts,Main Card,155,3 x 5,Brok Weaver,/fightcenter/bouts/502972-ufc-fight-night-roos...,/fightcenter/events/69127-ufc-fight-night,NaN,NaN
4,"Submission, Kneebar",2:36 Round 1 of 3,7,Mackenzie Dern,Main Card,115,3 x 5,Hannah Cifers,/fightcenter/bouts/500998-ufc-fight-night-mack...,/fightcenter/events/69127-ufc-fight-night,NaN,NaN
5,"Decision, Unanimous","3 Rounds, 15:00 Total",6,Katlyn Chookagian,Prelim,125,3 x 5,Antonina Shevchenko,/fightcenter/bouts/502988-ufc-fight-night-katl...,/fightcenter/events/69127-ufc-fight-night,NaN,NaN
6,"Decision, Unanimous","3 Rounds, 15:00 Total",5,Daniel Rodriguez,Prelim,170,3 x 5,Gabe Green,/fightcenter/bouts/503985-ufc-fight-night-dani...,/fightcenter/events/69127-ufc-fight-night,NaN,NaN
7,"KO/TKO, Knee to the Body to Ground and Pound",1:51 Round 1 of 3,4,Jamahal Hill,Prelim,205,3 x 5,Klidson Abreu,/fightcenter/bouts/500971-ufc-fight-night-jama...,/fightcenter/events/69127-ufc-fight-night,NaN,NaN
8,"Submission, Arm Triangle Choke","3:18 Round 2 of 3, 8:18 Total",3,Brandon Royval,Prelim,125,3 x 5,Tim Elliott,/fightcenter/bouts/502983-ufc-fight-night-tim-...,/fightcenter/events/69127-ufc-fight-night,NaN,NaN
9,"Submission, One-Arm Guillotine Choke",3:03 Round 1 of 3,2,Casey Kenney,Prelim,135,3 x 5,Louis Smolka,/fightcenter/bouts/502982-ufc-fight-night-case...,/fightcenter/events/69127-ufc-fight-night,NaN,NaN
